In [1]:
pip install openai jupyter python-dotenv -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

load_dotenv(find_dotenv(), override=True)
client = OpenAI()

In [ ]:
def create_meals(
    ingredients,
    kcal=2000,
    exact_ingredients=False,
    output_format="text",
    model="gpt-3.5-turbo",
    system_role="Você é um cozinheiro habilidoso com a expertise de um chef.",
    temperature=1,
    extra=None,
):
    prompt = f"""
    Crie um plano diário de alimentação para café da manhã, almoço e janta baseado nos seguintes ingedientes: 
    {ingredients}
    Seu output tem ser no formato {output_format}
     Siga as instruções abaixo cuidadosamente
     ### Instruções:
    1. {"Só use os ingredientes providos com sal, pimenta e especiarias." if exact_ingredients else "Fique livre para incorporar os ingredientes dados como base e adicione outros ingredientes se você considerar necessario para melhorar o sabor, valor nutricional ou apelo geral das receitas"}
    2. Especifique a quantidade exata de cada ingrediente
    3. Tenha certeza que a quantidade de calorias está abaixo de {kcal}
    4. Para cada refeição, explique cada refeição, passo a passo, com frases simples e claras. Use marcadores ou numeroes para organizar os passos
    5. Para cada refeição, especifique o total de calorias e o quantas pessoas servem
    6. Para cada refeição, dê um titulo conciso e descritivo que resuma os ingredientes principais e os sabores. O titulo deve ser um prompt valido pro DALL-E para gerar uma imagem original para a receita
    7. Para cada receita, diga o tempo total de preparo e cozinhamento.
    {"8. Se possivel as refeições devem ser:" + extra if extra else ""}

    Antes de responder, tenha certeza que seguiu as instruções listadas acima (pontos 1 a 7 ou 8)
    A ultima linha da sua resposta deve ser uma string que contém APENAS o titulo das refeições com uma virgula entre cada e nada mais
    \n Brocolis e ovo mexido, frango grelhado e vegetais, peixe cozido com salada de repolho
"""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
    )
    return response.choices[0].message.content

In [4]:
foods = "azeite extra virgem, grãos, frutas frescas e vegetais, nozes, peixe, ovo, comidas fermetadas, mel"
output = create_meals(
    ingredients=foods,
    model="gpt-4",
    output_format="HTML and CSS",
    extra="apimentado",
    exact_ingredients=False,
)
print(output)

<!DOCTYPE html>
<html>
<head>
<style>
body {
    font-family: Arial, sans-serif;
}
h2 {
    color: #4CAF50;
}
table {
    border-collapse: collapse;
    width: 100%;
}
th, td {
    padding: 15px;
    text-align: left;
    border-bottom: 1px solid #ddd;
}
</style>
</head>
<body>

<h2>Café da Manhã: Ovos Mexidos com Brócolis</h2>

<table>
  <tr>
    <th>Ingredientes</th>
    <th>Instruções de Preparo</th>
  </tr>
  <tr>
    <td>
      <ul>
        <li>2 ovos (140 cal)</li>
        <li>1 xícara de brócolis (55 cal)</li>
        <li>1 colher de sopa de azeite extra-virgem (120 cal)</li>
        <li>Pitada de pimenta (1 cal)</li>
      </ul>
    </td>
    <td>
      <ol>
        <li>Em uma frigideira, adicione o azeite e brócolis e cozinhe por 5 minutos. </li>
        <li>Em uma tigela, bata os ovos e despeje na frigideira. </li>
        <li>Mexa gentilmente até que os ovos estejam completamente cozidos. </li>
        <li>Adicione uma pitada de pimenta para dar um toque apimentado. </li>
  

In [5]:
# from IPython.display import display, HTML

# display(HTML(output)